<a href="https://colab.research.google.com/github/ritesh200tiwari/forest-fire-prediction/blob/main/chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#libraries needed for NLP
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

#Libararies needed for TensorFlow processing
import tensorflow as tf
import numpy as np
!pip install tflearn
import tflearn
import random
import json

     |████████████████████████████████| 112kB 6.7MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127300 sha256=345889fdaae2c56d10533fd94e0214208150aa05a896c9a049ae76b45ae40826
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn
Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],\n         "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],\n         "context": [""]\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],\n         "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],\n         "context": [""]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful", "Awesome, thanks", "Thanks for helping me"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"],\n         "context": [""]\n        },\n        {"tag": "noanswer",\n         "patterns": [],\n         "responses": ["Sorry, can\'t understand you", "Please give me more info", "Not sure I understand"],\n         "context"

In [9]:
#import chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [10]:
intents

{'intents': [{'context': [''],
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'context': [''],
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'context': [''],
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'context': [''],
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'tag': 'noanswer'},
  {'context': [''],
   'patterns': ['How you could help me?',
    'What help you pr

In [11]:
words = []
classes = []
documents = []
ignore = ['?']

#loop through each sentence in the intent's patters
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each and every word in the sentence
        w=nltk.word_tokenize(pattern)
        
        #add word to the word list
        words.extend(w)
        
        #add word(s) to documents
        documents.append((w, intent['tag']))
        
        #add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [13]:
#perform stemming and lower each word as well as remove duplicates

words=[stemmer.stem(w.lower()) for w in words if w not in ignore]
words=sorted(list(set(words)))

#remove duplicate classes
classes=sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

121 documents
43 classes ['HR_related_problem', 'Location', 'Weather', 'about', 'appointment status', 'cabin', 'check_leave', 'commission', 'competitors_in_market', 'configuration', 'connect_people', 'cost_lowering', 'customer_satisfaction', 'domain', 'email_id', 'factors_impacting_sale', 'forgot_password', 'gadgets', 'goodbye', 'greeting', 'highest_grossing', 'hours', 'invalid', 'key_customers', 'leave', 'maintainence', 'manufacturing_problems', 'missing_id', 'name', 'noans', 'options', 'order_components', 'order_tracking', 'predict_delay', 'predict_performance', 'project_handling_queries', 'search_department', 'search_person_by_id', 'solve_problems', 'supplier_info', 'thanks', 'turnover', 'version_update']
226 unique stemmed words ["'s", ',', '.', '23a12', '23a31', '32712', '345a23', '431b67', '561a24', '562b78', '@', 'a', 'abc', 'abx', 'accid', 'ai', 'am', 'an', 'ana', 'and', 'anyon', 'appoint', 'appoit', 'ar', 'at', 'awesom', 'bas', 'be', 'been', 'benefit', 'bhat', 'bor', 'bye', 'c

In [14]:
#create training data
training=[]
output=[]

#create an empty array for output
output_empty=[0]*len(classes)

#create training set bag of words for each sentence
for doc in documents:
  #initialize bag of words
  bag=[]
  #list of tokenized words for the pattern
  pattern_words=doc[0]
  #stemming each word
  pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
  #create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    
  #output is 1 for current tag and o for the rest of other tags
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]=1
    
  training.append([bag,output_row])
    
#shuffling features and turning it into np.array
random.shuffle(training)
training=np.array(training)
  
#creating training lists
train_x =list(training[:,0])
train_y =list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [16]:
#resetting underlying graph data
#tf.reset_default_graph()
tf.compat.v1.reset_default_graph


#building neural network
net=tflearn.input_data(shape=[None, len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]), activation='softmax')
net=tflearn.regression(net)

#defining model and setting up tensorboard
model=tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 15999  | total loss: 0.01045 | time: 0.071s
| Adam | epoch: 1000 | loss: 0.01045 - acc: 0.9871 -- iter: 120/121
Training Step: 16000  | total loss: 0.00944 | time: 0.075s
| Adam | epoch: 1000 | loss: 0.00944 - acc: 0.9884 -- iter: 121/121
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [17]:
import pickle
pickle.dump({'word':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open("training_data","wb"))

In [18]:
#restoring all the data structures
data = pickle.load(open("training_data","rb"))

words   =data['word']
classes =data['classes']
train_x =data['train_x']
train_y =data['train_y']

In [20]:
with open('intents.json') as json_data:
    intents = json.load(json_data)
  

In [21]:
#load the saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [22]:
def clean_up_sentence(sentence):
  #tokenize the token
  sentence_words=nltk.word_tokenize(sentence)
  #stemming each word
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

#return bag of word array:0 and 1 for each word in the bag that exists
def bow(sentence, words, show_details=False):
  #tokenize sentence
  sentence_words=clean_up_sentence(sentence)
  #generating bag of words
  bag=[0] * len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w ==s:
        bag[i] = 1
        
        if show_details:
          print("found in bag: %s" % w)
  return (np.array(bag))

In [23]:
ERROR_THRESHOLD = 0.0
def classify(sentence):
  #generate probabilities from the model
  results = model.predict([bow(sentence, words)])[0]
  
  #filter out prediction below a threshold
  results = [[i,r] for i, r in enumerate(results) if r> ERROR_THRESHOLD]
  
  #sort by strength of probability
  results.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  
  #return tuple of intent and probability
  return return_list

def response(sentence, userID='123', show_details=False):
  results=classify(sentence)
  
  if results:
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))
      results.pop(0)
  

In [24]:
classify('what are you hours of operation?')

[('gadgets', 0.66778),
 ('hours', 0.29498795),
 ('predict_performance', 0.036910806),
 ('turnover', 0.00025000906),
 ('Weather', 3.0431644e-05),
 ('highest_grossing', 2.1557133e-05),
 ('factors_impacting_sale', 1.5599451e-05),
 ('supplier_info', 3.3564938e-06),
 ('solve_problems', 2.030216e-07),
 ('cost_lowering', 4.672076e-08),
 ('search_department', 6.095831e-10),
 ('appointment status', 4.0614737e-10),
 ('competitors_in_market', 4.8854278e-11),
 ('customer_satisfaction', 1.9926374e-12),
 ('connect_people', 2.510803e-13),
 ('version_update', 3.4718752e-14),
 ('name', 1.3842153e-14),
 ('domain', 3.810664e-15),
 ('configuration', 4.528552e-17),
 ('commission', 3.388289e-17),
 ('search_person_by_id', 8.887698e-19),
 ('email_id', 3.812474e-19),
 ('key_customers', 1.1558047e-19),
 ('manufacturing_problems', 3.921492e-20),
 ('project_handling_queries', 4.307522e-21),
 ('options', 3.0356032e-21),
 ('goodbye', 2.513706e-21),
 ('check_leave', 1.3902034e-21),
 ('leave', 3.8336585e-22),
 ('Loca

In [25]:
response('Do you take credit cards?')

Can't Understand Your Question


In [26]:
response('Bye')

See you!


In [27]:
response('good day')

Hi there, how can I help?


In [28]:
response('Thank you')

Happy to help!


In [29]:
response('appointment status')

yes


In [30]:
response('Is Manish Kumar on leave?')

Yes On Leave
